<a href="https://colab.research.google.com/github/yaolihua081/NLP/blob/main/Final_Project_Oct2_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

from nltk.corpus import stopwords 
import seaborn as sns
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import sys, platform, numpy as np, nltk, re, sklearn, pandas as pd, seaborn as sns, collections
tmp = nltk.download(['stopwords', 'gutenberg','punkt'], quiet=True) # See https://www.nltk.org/book/ch02.html
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, HashingVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.datasets import fetch_20newsgroups
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from nltk.tokenize.casual import casual_tokenize
from nltk.tokenize import TweetTokenizer
from nltk import WordPunctTokenizer,RegexpTokenizer
from gensim.summarization.summarizer import summarize
from gensim.summarization import keywords as G_KEA
!pip install rake-nltk # https://pypi.org/project/rake-nltk/
from rake_nltk import Metric, Rake
# Increase viewable area of Pandas tables, numpy arrays, plots
pd.set_option('max_rows', 5, 'max_columns', 500, 'max_colwidth', 1, 'precision', 2)
np.set_printoptions(linewidth=10000, precision=4, edgeitems=20, suppress=True)

!pip install python-Levenshtein    # https://pypi.org/project/python-Levenshtein/0.12.0/
!pip install -U sentence-transformers   # https://github.com/UKPLab/sentence-transformers
!pip install umap-learn
import sys, platform, numpy as np, pandas as pd,  matplotlib.pylab as plt, matplotlib, pkg_resources, importlib.util
import re, sklearn, Levenshtein # nltk, 
from sentence_transformers import SentenceTransformer as SBERT


  Created wheel for rake-nltk: filename=rake_nltk-1.0.4-py2.py3-none-any.whl size=7819 sha256=3ab2054c70cd370ec2ee914f65da14bb3b3b699f65a23bbbecc8ae9667902cbc
  Stored in directory: /root/.cache/pip/wheels/ef/92/fc/271b3709e71a96ffe934b27818946b795ac6b9b8ff8682483f
Successfully built rake-nltk
     |████████████████████████████████| 51kB 2.9MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.0-cp36-cp36m-linux_x86_64.whl size=144802 sha256=ce190ec97122b13359df5d2239c7943b4701929721f06e993d3647565f65cc87
  Stored in directory: /root/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
Successfully built python-Levenshtein
     |████████████████████████████████| 71kB 4.1MB/s 
     |████████████████████████████████| 1.3MB 9.7MB/s 
     |████████████████████████████████| 1.1MB 43.4MB/s 
     |████████████████████████████████| 2.9MB 6.4MB/s 
     |████████████████████████████████| 890kB 41.8MB/s 
  Created wheel for sentence-transformers: 

In [2]:
#train_txt = pd.read_csv('train.csv') 
train_txt = pd.read_csv('https://raw.githubusercontent.com/yaolihua081/NLP/main/train.csv')
train_txt.dropna()
#test_txt = pd.read_csv('test.csv') 
test_txt = pd.read_csv('https://raw.githubusercontent.com/yaolihua081/NLP/main/test.csv')
test_txt.dropna()
train_txt['text'].values[314]='i do not know'
#line 314 was nan,


,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
...,...,...,...,...
27479,ed167662a5,But it was worth it ****.,But it was worth it ****.,positive
27480,6f7127d9d7,All this flirting going on - The ATG smiles. Yay. ((hugs)),All this flirting going on - The ATG smiles. Yay. ((hugs),neutral


,textID,text,sentiment
0,f87dea47db,Last session of the day http://twitpic.com/67ezh,neutral
1,96d74cb729,Shanghai is also really exciting (precisely -- skyscrapers galore). Good tweeps in China: (SH) (BJ).,positive
...,...,...,...
3532,df1baec676,_sutra what is your next youtube video gonna be about? I love your videos!,positive
3533,469e15c5a8,http://twitpic.com/4woj2 - omgssh ang cute ng bby.!,positive


0

In [3]:
def cleandata(train_txt):
  train_txt['text'] = train_txt['text'].str.strip()         # remove trailing and leading spaces
  train_txt['nChar'] = train_txt['text'].str.len()      # compute character length and save in a column
  BlankDocs_mask = train_txt.nChar <=1
  train_txt = train_txt[~BlankDocs_mask]         # ~ is the negation operator that flips between True & False
  #text=nan. df.dropna()
  return train_txt

train_txt=cleandata(train_txt)  
test_txt=cleandata(test_txt)

In [4]:
#combine positive, negative as one type, and create a colum 'label'
Neu_set=train_txt['sentiment']=='neutral'
Pos_set=train_txt['sentiment']=='positive'
Neg_set=train_txt['sentiment']=='negative'

train_txt['label']=train_txt['sentiment'].apply(lambda x: 0 if x=='neutral' else 1 )
test_txt['label']=test_txt['sentiment'].apply(lambda x: 0 if x=='neutral' else 1 )

#get size
[sum(test_txt['label']==1),sum(test_txt['label']==0)]
[sum(train_txt['label']==1),sum(train_txt['label']==0)]


[2104, 1430]

[16363, 11117]

In [5]:

def RakeExtract(x):
  r = Rake()#will change parameters later
  #keywords=set(x)
  if (len(x.split())>1):  
    r.extract_keywords_from_text(x) 
    dfRKS0=r.get_ranked_phrases_with_scores()
    column_names=['scores','kwd']
    dfRKS0=pd.DataFrame(dfRKS0,columns=column_names)
    keywords=set(dfRKS0['kwd'].tolist())
  else:
     keywords=set(x)     
  return keywords  
train_txt['Rake_Extracted'] = train_txt['text'].apply(lambda x: RakeExtract(x))
train_txt
test_txt['Rake_Extracted'] = test_txt['text'].apply(lambda x: RakeExtract(x))

,textID,text,selected_text,sentiment,nChar,label,Rake_Extracted
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,35,0,"{going, responded}"
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,45,1,"{miss, sooo sad, san diego !!}"
...,...,...,...,...,...,...,...
27479,ed167662a5,But it was worth it ****.,But it was worth it ****.,positive,26,1,"{worth, ****.}"
27480,6f7127d9d7,All this flirting going on - The ATG smiles. Yay. ((hugs)),All this flirting going on - The ATG smiles. Yay. ((hugs),neutral,59,0,"{atg smiles, yay, (( hugs )), flirting going}"


In [ ]:
#summarize(x,ratio=2) must have more than one sentences. so not work for tweets
def KEAExtracted(x,ratio=1):
  if (len(x.split())>1): 
    dfGK=G_KEA(x, ratio=ratio,lemmatize=True,scores=True) #str
    column_names=['kwd','scores']
    dfGK=pd.DataFrame(dfGK,columns=column_names)
    keywords=set(dfGK['kwd'].tolist())
  else:
     keywords=set(x)     
  return keywords  

train_txt['KEA_Extracted'] = train_txt['text'].apply(lambda x: KEAExtracted(x))
train_txt
test_txt['KEA_Extracted'] = test_txt['text'].apply(lambda x: KEAExtracted(x))  

,textID,text,selected_text,sentiment,nChar,label,Rake_Extracted,KEA_Extracted
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,35,0,"{going, responded}",{}
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,45,1,"{miss, san diego !!, sooo sad}","{sad, san, sooo, diego}"
...,...,...,...,...,...,...,...,...
27479,ed167662a5,But it was worth it ****.,But it was worth it ****.,positive,26,1,"{worth, ****.}",{}
27480,6f7127d9d7,All this flirting going on - The ATG smiles. Yay. ((hugs)),All this flirting going on - The ATG smiles. Yay. ((hugs),neutral,59,0,"{(( hugs )), flirting going, atg smiles, yay}","{yay, hugs, atg, smiles, flirting going}"


In [5]:
def TrainTest(tarin_txt, test_txt, name, Label='label'):
  #tY=train_txt['sentiment']
  tY=train_txt[Label]
  tTxt=train_txt[name]#name='text','selected_txt', 'Extracted'
  #tTxt=train_txt['selected_text']#use this one the scores gets smaller
  tY=tY.tolist()
  #vY=test_txt['sentiment']
  vY=test_txt[Label]
  if name=='selected_text':
    vTxt=test_txt['text']
  else:
    vTxt=test_txt[name]

  vY=vY.tolist()
  tX_row_names=train_txt['textID']
  vX_row_names=test_txt['textID']
  return tTxt, tY, tX_row_names, vTxt, vY, vX_row_names

#tTxt, tY, tX_row_names, vTxt, vY, vX_row_names= TrainTest(train_txt, test_txt, 'Rake_Extracted')

In [6]:
def PreModel(train_txt,test_txt, stopword_list,name, TF_IDF=True, Label='label'):
  #stopword_list = nltk.corpus.stopwords.words('english')
  tTxt, tY, tX_row_names, vTxt, vY, vX_row_names= TrainTest(train_txt, test_txt, name, Label)
  CntVec=CountVectorizer(stop_words=stopword_list,token_pattern=r'(?u)\b\w+\b', lowercase=True, max_features=1000)
  tX = CntVec.fit_transform(tTxt.values.astype('U')) 
  vX = CntVec.transform(vTxt.values.astype('U')) 
  if TF_IDF:
    tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
    tX=tfidf_transformer.fit_transform(tX)
    vX=tfidf_transformer.transform(vX)
  column_names=CntVec.get_feature_names()
  tX=tX.toarray()
  vX=vX.toarray()
  tX = pd.DataFrame(tX, columns=column_names, index=tX_row_names)
  vX = pd.DataFrame(vX, columns=column_names, index=vX_row_names)
  
  return tX,tY, vX,vY

#stopword_list = nltk.corpus.stopwords.words('english')
#PreModel(train_txt,test_txt, stopword_list,'Rake_Extracted')


In [7]:
#Logistic regression using Rake_extracted
def ML(model, train_txt, test_txt,name, stopword_list, TF_IDF=True, Label='label'):
  tX, tY,vX, vY=PreModel(train_txt,test_txt, stopword_list,name, TF_IDF, Label)
  if (model=='LRG'):
    para_seed = [(c, iter) for c in np.linspace(.01, 1, num=50) for iter in np.linspace(10, 50, num=5)]
    lvc_df = pd.DataFrame(para_seed, columns=['c', 'iter'])
    lvc_df['model'] =  lvc_df.apply(lambda row: LogisticRegression(n_jobs=-1, C=row.c, max_iter=row.iter).fit(tX, list(tY)), axis=1)
    #lvc_df['score'] = lvc_df['model'].apply(lambda model: model.score(vX, list(vY)))
  if (model=='LVC'):
    c_seed = np.linspace(.05, 2, num=100)
    lvc_df = pd.DataFrame(c_seed, columns=['c'])
    lvc_df['model'] = lvc_df['c'].apply(lambda c: LinearSVC(C=c).fit(tX, list(tY)))
    #lvc_df['score'] = lvc_df['model'].apply(lambda model: model.score(TvX, list(vY)))

  if (model=='RF'):
    n_estimators = range(10, 200,100)
    lvc_df = pd.DataFrame(n_estimators, columns=['c'])
    lvc_df['model']=lvc_df['c'].apply(lambda c: RandomForestClassifier(n_estimators=c).fit(tX, list(tY)))
  if (model=='NB'):
    alphas = np.linspace(.1, 100, num=100)
    lvc_df = pd.DataFrame(alphas, columns=['c'])
    lvc_df['model'] = lvc_df['c'].apply(lambda c: MultinomialNB(alpha=c).fit(tX, list(tY)))
    
  lvc_df['score']=lvc_df['model'].apply(lambda model: model.score(vX, list(vY)))
  filter=lvc_df['score']==max(lvc_df['score'])
  bestmodel=lvc_df[filter]['model'].values[0]
  train_score=bestmodel.score(tX,list(tY))

  return bestmodel, train_score, max(lvc_df['score'])
  #return max(lvc_df['score'])

In [10]:
#TF-IDF=True or False, 
#change name to be different,
#first use Label='sentiment' to run all train_text
stopword_list = nltk.corpus.stopwords.words('english')
name='text'
#name='Rake_Extracted'
#name='selected_text'
LRGRate_model, LGRRate_train_score, LGRRate_test_score=ML('LRG', train_txt, test_txt,name, stopword_list,TF_IDF=True, Label='sentiment')
NBRate_model, NBRate_train_score, NBRate_test_score=ML('NB', train_txt, test_txt,name, stopword_list,TF_IDF=True, Label='sentiment')
LVCRate_model, LVCRate_train_score, LVCRate_test_score=ML('LVC', train_txt, test_txt,name, stopword_list,TF_IDF=True, Label='sentiment')  
RFRate_model, RFRate_train_score, RFRate_test_score=ML('RF', train_txt, test_txt,name, stopword_list,TF_IDF=True, Label='sentiment')

LRGRate_model, LGRRate_train_score, LGRRate_test_score
NBRate_model, NBRate_train_score, NBRate_test_score

LVCRate_model, LVCRate_train_score, LVCRate_test_score

RFRate_model, RFRate_train_score, RFRate_test_score

(LogisticRegression(C=0.7979591836734694, class_weight=None, dual=False,
                    fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                    max_iter=30.0, multi_class='auto', n_jobs=-1, penalty='l2',
                    random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                    warm_start=False), 0.7121542940320232, 0.691284663271081)

(MultinomialNB(alpha=0.1, class_prior=None, fit_prior=True),
 0.6675764192139738,
 0.6544991511035654)

(LinearSVC(C=0.22727272727272724, class_weight=None, dual=True,
           fit_intercept=True, intercept_scaling=1, loss='squared_hinge',
           max_iter=1000, multi_class='ovr', penalty='l2', random_state=None,
           tol=0.0001, verbose=0), 0.7118995633187772, 0.6918505942275043)

(RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                        criterion='gini', max_depth=None, max_features='auto',
                        max_leaf_nodes=None, max_samples=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=1, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, n_estimators=110,
                        n_jobs=None, oob_score=False, random_state=None,
                        verbose=0, warm_start=False),
 0.9676855895196507,
 0.687889077532541)

In [ ]:
#TF-IDF=True or False, Label='label' 
#change name to be different,
#second  use Label='label' to run all train_text, but with two classes.
stopword_list = nltk.corpus.stopwords.words('english')
name='text'
#name='Rake_Extracted'
#name='selected_text'
LRGRate_model, LGRRate_train_score, LGRRate_test_score=ML('LRG', train_txt, test_txt,name, stopword_list,TF_IDF=True, Label='label')
NBRate_model, NBRate_train_score, NBRate_test_score=ML('NB', train_txt, test_txt,name, stopword_list,TF_IDF=True, Label='label')
LVCRate_model, LVCRate_train_score, LVCRate_test_score=ML('LVC', train_txt, test_txt,name, stopword_list,TF_IDF=True, Label='label')  
RFRate_model, RFRate_train_score, RFRate_test_score=ML('RF', train_txt, test_txt,name, stopword_list,TF_IDF=True, Label='label')

LRGRate_model, LGRRate_train_score, LGRRate_test_score
NBRate_model, NBRate_train_score, NBRate_test_score

LVCRate_model, LVCRate_train_score, LVCRate_test_score

RFRate_model, RFRate_train_score, RFRate_test_score

In [ ]:
#random sample Neu-set to have sample size 8000 to be balance
#[sum(Neu_set),sum(Pos_set), sum(Neg_set)]
#from random import random
#import random
#nums=random.sample(range(11117),8000)
#temp=pd.DataFrame(train_txt[Neu_set])
#trainNeu=temp.iloc[nums]
#train_txt=pd.concat([train_txt[Neg_set],train_txt[Pos_set],trainNeu])


In [43]:
tX.shape

(27480, 1000)

In [8]:
stopword_list = nltk.corpus.stopwords.words('english')
name='text'

Label='label'
tX, tY,vX, vY=PreModel(train_txt,test_txt, stopword_list,name, Label=Label)
#9/25/20
#will compare this vecter with BERT and  Labse produced vector! then run LG, RF model
#will try RNN , CNN model

In [ ]:
tX.Y=pd.concat([tX,pd.DataFrame(tY)])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """Entry point for launching an IPython kernel.


BERT Extraction

In [9]:
!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer as SBERT
from sklearn.metrics.pairwise import cosine_similarity
mSBERT = SBERT('xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

Requirement already up-to-date: sentence-transformers in /usr/local/lib/python3.6/dist-packages (0.3.9)


100%|██████████| 1.01G/1.01G [00:56<00:00, 17.9MB/s]


In [ ]:

dfBERT_train=pd.DataFrame(mSBERT.encode(train_txt['text'].tolist()), index=train_txt['textID'])
dfBERT_test=pd.DataFrame(mSBERT.encode(test_txt['text'].tolist()), index=test_txt['textID'])
#Bert data
tX_numpy=dfBERT_train.to_numpy()
tY_numpy=np.array(tY)
vX_numpy=dfBERT_test.to_numpy()
vY_numpy=np.array(vY)

In [13]:
def BERTML(model, dfBERT_train, tY, dfBERT_test, vY):
  
  if (model=='LRG'):
    para_seed = [(c, iter) for c in np.linspace(.01, 1, num=50) for iter in np.linspace(10, 50, num=5)]
    lvc_df = pd.DataFrame(para_seed, columns=['c', 'iter'])
    lvc_df['model'] =  lvc_df.apply(lambda row: LogisticRegression(n_jobs=-1, C=row.c, max_iter=row.iter).fit(dfBERT_train, list(tY)), axis=1)
    #lvc_df['score'] = lvc_df['model'].apply(lambda model: model.score(vX, list(vY)))
  if (model=='LVC'):
    c_seed = np.linspace(.05, 2, num=100)
    lvc_df = pd.DataFrame(c_seed, columns=['c'])
    lvc_df['model'] = lvc_df['c'].apply(lambda c: LinearSVC(C=c).fit(dfBERT_train, list(tY)))
    #lvc_df['score'] = lvc_df['model'].apply(lambda model: model.score(TvX, list(vY)))

  if (model=='RF'):
    n_estimators = range(10, 200,100)
    lvc_df = pd.DataFrame(n_estimators, columns=['c'])
    lvc_df['model']=lvc_df['c'].apply(lambda c: RandomForestClassifier(n_estimators=c).fit(dfBERT_train, list(tY)))
  if (model=='NB'):
    alphas = np.linspace(.1, 100, num=100)
    lvc_df = pd.DataFrame(alphas, columns=['c'])
    lvc_df['model'] = lvc_df['c'].apply(lambda c: MultinomialNB(alpha=c).fit(dfBERT_train, list(tY)))
    
  lvc_df['score']=lvc_df['model'].apply(lambda model: model.score(dfBERT_test, list(vY)))
  filter=lvc_df['score']==max(lvc_df['score'])
  bestmodel=lvc_df[filter]['model'].values[0]
  train_score=bestmodel.score(dfBERT_train,list(tY))

  return bestmodel, train_score, max(lvc_df['score'])
  #return max(lvc_df['score'])

In [ ]:
model="LVC"

LVCModel, LVCTrainScore, LVCscore=BERTML(model, dfBERT_train, tY, dfBERT_test, vY)

model="RF"
RFModel, RFTrainScore, RFscore=BERTML(model, dfBERT_train, tY, dfBERT_test, vY)



model="LRG"
LRGModel, LRGTrainScore, LRGscore=BERTML(model, dfBERT_train, tY, dfBERT_test, vY)

final=[[LVCModel, LVCTrainScore, LVCscore],[LRGModel, LRGTrainScore, LRGscore],[RFModel, RFTrainScore, RFscore]]
final

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, SimpleRNN, Flatten
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Conv1D, MaxPooling1D
import pandas as pd

max_review_length = 500
vocabulary_size = 500
embedding_dim=500#
# Fully connected neural network
model_ffn = Sequential()
model_ffn.add(Dense(250, activation='relu',input_dim=max_review_length))
model_ffn.add(Dense(1, activation='sigmoid'))
model_ffn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

history_ffn = model_ffn.fit([tX, pd.DataFrame(tY)])
#(tX, tY, epochs=20, batch_size=128, verbose=2)
#history_ffn = model_ffn.fit(tX, tY, verbose=2)
#scores_ffn = model_ffn.evaluate(vX, vY, verbose=0)

# Fully connected neural network with embeddings
model_ffne = Sequential()
model_ffne.add(Embedding(vocabulary_size, embedding_dim, input_length=max_review_length))
model_ffne.add(Flatten())
model_ffne.add(Dense(250, activation='relu'))
model_ffne.add(Dense(1, activation='sigmoid'))
model_ffne.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#history_ffne = model_ffne.fit(tX, tY, epochs=20, batch_size=128, verbose=2)
#scores_ffne = model_ffne.evaluate(vX, vY, verbose=0)

# Convolutional neural network
model_cnn = Sequential()
model_cnn.add(Embedding(vocabulary_size, embedding_dim, input_length=max_review_length))
model_cnn.add(Conv1D(filters=200, kernel_size=3, padding='same', activation='relu'))
model_cnn.add(MaxPooling1D(pool_size=2))
model_cnn.add(Flatten())
model_cnn.add(Dense(250, activation='relu'))
model_cnn.add(Dense(1, activation='sigmoid'))
model_cnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history_cnn = model_cnn.fit(tX, tY, verbose=2)
scores_cnn = model_cnn.evaluate(vX, vY, verbose=0)

# Recurrent Neural Network
model_rnn = Sequential()
model_rnn.add(Embedding(vocabulary_size, embedding_dim, input_length=max_review_length))
model_rnn.add(SimpleRNN(100))
model_rnn.add(Dense(1, activation='sigmoid'))
model_rnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history_rnn = model_rnn.fit(tX, tY, epochs=3, batch_size=64)
scores_rnn = model_rnn.evaluate(vX, vY, verbose=0)

# Convolutional + Recurrent Neural Network
model_cnn_rnn = Sequential()
model_cnn_rnn.add(Embedding(vocabulary_size, embedding_dim, input_length=max_review_length))
model_cnn_rnn.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model_cnn_rnn.add(MaxPooling1D(pool_size=2))
model_cnn_rnn.add(SimpleRNN(100))
model_cnn_rnn.add(Dense(1, activation='sigmoid'))
model_cnn_rnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history_cnn_rnn = model_cnn_rnn.fit(tX, tY, epochs=10, batch_size=64)
scores_cnn_rnn = model_cnn_rnn.evaluate(vX, vY, verbose=0)

# Evaluation
df = pd.DataFrame({'Model': ['FFNN', 'FFNN with Embeddings', 'CNN', 'RNN', 'CNN+RNN'],
                   'Accuracy': [str(round(scores_ffn[1]*100, 2)) + '%',
                                str(round(scores_ffne[1]*100, 2)) + '%',
                                str(round(scores_cnn[1]*100, 2)) + '%',
                                str(round(scores_rnn[1]*100, 2)) + '%',
                                str(round(scores_cnn_rnn[1]*100, 2)) + '%']})
df

In [14]:
stopword_list = nltk.corpus.stopwords.words('english')
name='text'

Label='label'
tX, tY,vX, vY=PreModel(train_txt,test_txt, stopword_list,name, Label=Label)
tX_numpy=tX.to_numpy()
tY_numpy=np.array(tY)
vX_numpy=vX.to_numpy()
vY_numpy=np.array(vY)

In [15]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, SimpleRNN, Flatten
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Conv1D, MaxPooling1D
import pandas as pd

max_review_length = 1000
vocabulary_size = 1000
embedding_dim=1000#
embedding_dim=50
# Fully connected neural network
model_ffn = Sequential()
model_ffn.add(Dense(250, activation='relu',input_dim=max_review_length))
model_ffn.add(Dense(1, activation='sigmoid'))
model_ffn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#history_ffn = model_ffn.fit(X_train, y_train,epochs=20, batch_size=128, verbose=2)
#history_ffn = model_ffn.fit([tX, pd.DataFrame(tY)])

history_ffn = model_ffn.fit(tX_numpy, tY_numpy, epochs=30,verbose=2)

scores_ffn = model_ffn.evaluate(vX_numpy, vY_numpy, verbose=0)
scores_ffn

Epoch 1/30
859/859 - 2s - loss: 0.6022 - accuracy: 0.6754
Epoch 2/30
859/859 - 2s - loss: 0.5476 - accuracy: 0.7378
Epoch 3/30
859/859 - 2s - loss: 0.5240 - accuracy: 0.7555
Epoch 4/30
859/859 - 2s - loss: 0.4994 - accuracy: 0.7680
Epoch 5/30
859/859 - 2s - loss: 0.4709 - accuracy: 0.7882
Epoch 6/30
859/859 - 2s - loss: 0.4348 - accuracy: 0.8087
Epoch 7/30
859/859 - 2s - loss: 0.3919 - accuracy: 0.8344
Epoch 8/30
859/859 - 2s - loss: 0.3450 - accuracy: 0.8604
Epoch 9/30
859/859 - 2s - loss: 0.2956 - accuracy: 0.8874
Epoch 10/30
859/859 - 2s - loss: 0.2496 - accuracy: 0.9129
Epoch 11/30
859/859 - 2s - loss: 0.2124 - accuracy: 0.9297
Epoch 12/30
859/859 - 2s - loss: 0.1825 - accuracy: 0.9410
Epoch 13/30
859/859 - 2s - loss: 0.1600 - accuracy: 0.9491
Epoch 14/30
859/859 - 2s - loss: 0.1439 - accuracy: 0.9526
Epoch 15/30
859/859 - 2s - loss: 0.1319 - accuracy: 0.9557
Epoch 16/30
859/859 - 2s - loss: 0.1234 - accuracy: 0.9564
Epoch 17/30
859/859 - 3s - loss: 0.1166 - accuracy: 0.9580
Epoch 

[1.019672155380249, 0.7037351727485657]

In [17]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, SimpleRNN, Flatten
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Conv1D, MaxPooling1D
import pandas as pd


max_review_length = 768
vocabulary_size = 768
max_review_length = 1000
vocabulary_size = 1000
embedding_dim=50#
# Convolutional neural network
model_cnn = Sequential()
model_cnn.add(Embedding(vocabulary_size, embedding_dim, input_length=max_review_length))
model_cnn.add(Conv1D(filters=200, kernel_size=3, padding='same', activation='relu'))
model_cnn.add(MaxPooling1D(pool_size=2))
model_cnn.add(Flatten())
model_cnn.add(Dense(250, activation='relu'))
model_cnn.add(Dense(1, activation='sigmoid'))
model_cnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history_cnn = model_cnn.fit(tX_numpy, tY_numpy,epochs=30, batch_size=128,verbose=2)
scores_cnn = model_cnn.evaluate(vX_numpy, vY_numpy, verbose=0)
scores_cnn

Epoch 1/30
215/215 - 260s - loss: 0.6769 - accuracy: 0.5997
Epoch 2/30
215/215 - 260s - loss: 0.6622 - accuracy: 0.6161
Epoch 3/30
215/215 - 263s - loss: 0.6548 - accuracy: 0.6225
Epoch 4/30
215/215 - 260s - loss: 0.6528 - accuracy: 0.6234
Epoch 5/30
215/215 - 260s - loss: 0.6521 - accuracy: 0.6231
Epoch 6/30
215/215 - 262s - loss: 0.6500 - accuracy: 0.6242
Epoch 7/30
215/215 - 265s - loss: 0.6493 - accuracy: 0.6239
Epoch 8/30
215/215 - 262s - loss: 0.6483 - accuracy: 0.6248
Epoch 9/30
215/215 - 267s - loss: 0.6477 - accuracy: 0.6248
Epoch 10/30
215/215 - 262s - loss: 0.6473 - accuracy: 0.6250
Epoch 11/30
215/215 - 263s - loss: 0.6469 - accuracy: 0.6248
Epoch 12/30
215/215 - 262s - loss: 0.6467 - accuracy: 0.6253
Epoch 13/30
215/215 - 263s - loss: 0.6470 - accuracy: 0.6249
Epoch 14/30
215/215 - 266s - loss: 0.6464 - accuracy: 0.6249
Epoch 15/30
215/215 - 263s - loss: 0.6462 - accuracy: 0.6252
Epoch 16/30
215/215 - 266s - loss: 0.6463 - accuracy: 0.6251
Epoch 17/30
215/215 - 262s - loss

[0.6925280094146729, 0.6117713451385498]

In [20]:
embedding_dim=100#

# Recurrent Neural Network
model_rnn = Sequential()
model_rnn.add(Embedding(vocabulary_size, embedding_dim, input_length=max_review_length))
model_rnn.add(SimpleRNN(100))
model_rnn.add(Dense(1, activation='sigmoid'))
model_rnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history_rnn = model_rnn.fit(tX_numpy, tY_numpy,epochs=30, batch_size=128,verbose=2)
scores_rnn = model_rnn.evaluate(vX_numpy, vY_numpy, verbose=0)
scores_rnn

Epoch 1/30
215/215 - 150s - loss: 0.6766 - accuracy: 0.5951
Epoch 2/30
215/215 - 149s - loss: 0.6760 - accuracy: 0.5955
Epoch 3/30
215/215 - 152s - loss: 0.6777 - accuracy: 0.5949
Epoch 4/30
215/215 - 150s - loss: 0.6766 - accuracy: 0.5955
Epoch 5/30
215/215 - 152s - loss: 0.6762 - accuracy: 0.5955
Epoch 6/30
215/215 - 149s - loss: 0.6759 - accuracy: 0.5955
Epoch 7/30
215/215 - 149s - loss: 0.6764 - accuracy: 0.5955
Epoch 8/30
215/215 - 150s - loss: 0.6765 - accuracy: 0.5955
Epoch 9/30
215/215 - 151s - loss: 0.6763 - accuracy: 0.5955
Epoch 10/30
215/215 - 151s - loss: 0.6757 - accuracy: 0.5955
Epoch 11/30
215/215 - 151s - loss: 0.6759 - accuracy: 0.5955
Epoch 12/30
215/215 - 151s - loss: 0.6756 - accuracy: 0.5955
Epoch 13/30
215/215 - 151s - loss: 0.6762 - accuracy: 0.5955
Epoch 14/30
215/215 - 150s - loss: 0.6752 - accuracy: 0.5955
Epoch 15/30
215/215 - 150s - loss: 0.6760 - accuracy: 0.5955
Epoch 16/30
215/215 - 151s - loss: 0.6761 - accuracy: 0.5955
Epoch 17/30
215/215 - 149s - loss

[0.6766024231910706, 0.5953593850135803]

In [ ]:
#11/1/2020
from keras.models import Sequential
from keras.layers import Dense, LSTM, SimpleRNN, Flatten
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Conv1D, MaxPooling1D
import pandas as pd

max_review_length = 1000
vocabulary_size = 1000
embedding_dim=500#

# Fully connected neural network
model_ffn = Sequential()
model_ffn.add(Dense(250, activation='relu',input_dim=max_review_length))
model_ffn.add(Dense(1, activation='sigmoid'))
model_ffn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

history_ffn = model_ffn.fit(tX_numpy, tY_numpy, epochs=30,verbose=2)
scores_ffn = model_ffn.evaluate(vX_numpy, vY_numpy, verbose=0)

# Fully connected neural network with embeddings
model_ffne = Sequential()
model_ffne.add(Embedding(vocabulary_size, embedding_dim, input_length=max_review_length))
model_ffne.add(Flatten())
model_ffne.add(Dense(250, activation='relu'))
model_ffne.add(Dense(1, activation='sigmoid'))
model_ffne.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

history_ffne = model_ffne.fit(tX_numpy, tY_numpy, epochs=30, batch_size=128, verbose=2)
scores_ffne = model_ffne.evaluate(vX_numpy, vY_numpy, verbose=0)


# Convolutional neural network
model_cnn = Sequential()
model_cnn.add(Embedding(vocabulary_size, embedding_dim, input_length=max_review_length))
model_cnn.add(Conv1D(filters=200, kernel_size=3, padding='same', activation='relu'))
model_cnn.add(MaxPooling1D(pool_size=2))
model_cnn.add(Flatten())
model_cnn.add(Dense(250, activation='relu'))
model_cnn.add(Dense(1, activation='sigmoid'))
model_cnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history_cnn = model_cnn.fit(tX_numpy, tY_numpy,epochs=30, batch_size=128,verbose=2)
scores_cnn = model_cnn.evaluate(vX_numpy, vY_numpy, verbose=0)
# Recurrent Neural Network
model_rnn = Sequential()
model_rnn.add(Embedding(vocabulary_size, embedding_dim, input_length=max_review_length))
model_rnn.add(SimpleRNN(100))
model_rnn.add(Dense(1, activation='sigmoid'))
model_rnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history_rnn = model_rnn.fit(tX_numpy, tY_numpy,epochs=30, batch_size=128,verbose=2)
scores_rnn = model_rnn.evaluate(vX_numpy, vY_numpy, verbose=0)

# Convolutional + Recurrent Neural Network

model_cnn_rnn = Sequential()
model_cnn_rnn.add(Embedding(vocabulary_size, embedding_dim, input_length=max_review_length))
model_cnn_rnn.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model_cnn_rnn.add(MaxPooling1D(pool_size=2))
model_cnn_rnn.add(SimpleRNN(100))
model_cnn_rnn.add(Dense(1, activation='sigmoid'))
model_cnn_rnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history_cnn_rnn = model_cnn_rnn.fit(tX_numpy, tY_numpy,epochs=30, batch_size=128,verbose=2)
scores_cnn_rnn = model_cnn_rnn.evaluate(vX_numpy, vY_numpy, verbose=0)
scores_cnn_rnn
# Evaluation
df = pd.DataFrame({'Model': ['FFNN', 'FFNN with Embeddings', 'CNN', 'RNN', 'CNN+RNN'],
                   'Accuracy': [str(round(scores_ffn[1]*100, 2)) + '%',
                                str(round(scores_ffne[1]*100, 2)) + '%',
                                str(round(scores_cnn[1]*100, 2)) + '%',
                                str(round(scores_rnn[1]*100, 2)) + '%',
                                str(round(scores_cnn_rnn[1]*100, 2)) + '%']})
df

Epoch 1/30
859/859 - 2s - loss: 0.6025 - accuracy: 0.6774
Epoch 2/30
859/859 - 2s - loss: 0.5479 - accuracy: 0.7369
Epoch 3/30
859/859 - 2s - loss: 0.5238 - accuracy: 0.7532
Epoch 4/30
859/859 - 2s - loss: 0.4968 - accuracy: 0.7695
Epoch 5/30
859/859 - 2s - loss: 0.4650 - accuracy: 0.7908
Epoch 6/30
859/859 - 2s - loss: 0.4251 - accuracy: 0.8125
Epoch 7/30
859/859 - 2s - loss: 0.3798 - accuracy: 0.8421
Epoch 8/30
859/859 - 3s - loss: 0.3316 - accuracy: 0.8686
Epoch 9/30
859/859 - 2s - loss: 0.2843 - accuracy: 0.8945
Epoch 10/30
859/859 - 2s - loss: 0.2392 - accuracy: 0.9175
Epoch 11/30
859/859 - 2s - loss: 0.2029 - accuracy: 0.9346
Epoch 12/30
859/859 - 2s - loss: 0.1757 - accuracy: 0.9422
Epoch 13/30
859/859 - 2s - loss: 0.1550 - accuracy: 0.9497
Epoch 14/30
859/859 - 2s - loss: 0.1397 - accuracy: 0.9541
Epoch 15/30
859/859 - 2s - loss: 0.1291 - accuracy: 0.9567
Epoch 16/30
859/859 - 2s - loss: 0.1199 - accuracy: 0.9586
Epoch 17/30
859/859 - 2s - loss: 0.1140 - accuracy: 0.9594
Epoch 